In [2]:
library("sleuth")
library("dplyr")
library("ggplot2")
library("tidyverse")

In [28]:
## initiate paths

root_path <- '/gladstone/ott/home/izheludev/denim/dev_011525'
sample_ids <- dir(file.path(root_path, "06_bootstrap"))
data_path <- file.path(root_path, "06_bootstrap", sample_ids)

metadata_filename <- "metadata.tsv"
bootstrap_dirname <- "06_bootstrap"

In [29]:
## load metadata and append paths

metadata <- read.table(file.path(root_path, metadata_filename),
                       header = TRUE,
                       stringsAsFactors=TRUE) %>%
            mutate(path = file.path(root_path, bootstrap_dirname, sample))

In [5]:
## set variables

pseudocount <- 0.5
alpha <- 0.05
FC_threshold <- 10

In [32]:
## set filenames

counts_filename <- "normalized_estimated_counts.tsv"
tpm_filename <- "normalized_tpm.tsv"
wald_filename <- "Wald_results.tsv"
lrt_filename <- "LRT_results.tsv"
plotting_filename <- "plotting.tsv"
wald_plot_filename <- "Wald_volcano.png"
lrt_plot_filename <- "LRT_volcano.png"

In [8]:
## declare design formula

parameter <- "strain"
design_formula <- reformulate(parameter)

In [9]:
## initialize sleuth object

sleuth_object <- sleuth_prep(metadata,
                             extra_bootstrap_summary = TRUE,
                             read_bootstrap_tpm = TRUE)

reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.


normalizing est_counts

656 targets passed the filter

normalizing tpm

merging in metadata

summarizing bootstraps





In [10]:
## fit full model

sleuth_object <- sleuth_fit(sleuth_object,
                            design_formula,
                            fit_name = "full")

fitting measurement error models

shrinkage estimation

computing variance of betas



In [11]:
## fit reduced model

sleuth_object <- sleuth_fit(sleuth_object,
                            ~1,
                            fit_name = "reduced")

fitting measurement error models

shrinkage estimation

1 NA values were found during variance shrinkage estimation due to mean observation values outside of the range used for the LOESS fit.
The LOESS fit will be repeated using exact computation of the fitted surface to extrapolate the missing values.
These are the target ids with NA values: s_156

computing variance of betas



In [12]:
## perform likelihood ratio test

sleuth_object <- sleuth_lrt(sleuth_object,
                            "reduced",
                            "full")

In [13]:
## Capture the models output
model_info <- capture.output(models(sleuth_object))

## Find the coefficient lines after "coefficients:" for the full model
coef_start <- grep("coefficients:", model_info)[1]  ## Get first occurrence (full model)
key_coef <- trimws(gsub("\\t", "", model_info[coef_start + 2]))

In [14]:
## perform Wald test

sleuth_object <- sleuth_wt(sleuth_object,
                           key_coef,
                           which_model = "full")

In [15]:
## extract results

wald_results <- sleuth_results(sleuth_object,
                               key_coef,
                               "wt",
                               show_all = TRUE)

lrt_results <- sleuth_results(sleuth_object,
                               "reduced:full",
                               "lrt",
                               show_all = TRUE)

In [16]:
## save counts results

sleuth_to_matrix(sleuth_object,
                 "obs_norm",
                 "est_counts") %>%
as_tibble(rownames = "target_id") %>%
arrange(str_extract(target_id, "^[^0-9]+"),
        as.numeric(str_extract(target_id, "\\d+"))) %>%
write_tsv(counts_filename)

Warning message:
“`select_()` was deprecated in dplyr 0.7.0.
ℹ Please use `select()` instead.
ℹ The deprecated feature was likely used in the sleuth package.
  Please report the issue at
  <https://github.com/pachterlab/sleuth/issues>.”
Warning message:
“`spread_()` was deprecated in tidyr 1.2.0.
ℹ Please use `spread()` instead.
ℹ The deprecated feature was likely used in the sleuth package.
  Please report the issue at
  <https://github.com/pachterlab/sleuth/issues>.”


In [31]:
## save tpm results

sleuth_to_matrix(sleuth_object,
                 "obs_norm",
                 "tpm") %>%
as_tibble(rownames = "target_id") %>%
arrange(str_extract(target_id, "^[^0-9]+"),
        as.numeric(str_extract(target_id, "\\d+"))) %>%
write_tsv(tpm_filename)

In [17]:
## save statistical results

wald_results %>% write_tsv(wald_filename)
lrt_results %>% write_tsv(lrt_filename)

In [33]:
## ordered unique groups from metadata
groups <- metadata %>%
  distinct(!!sym(parameter)) %>%
  pull()

## get plotting table
plotting <- sleuth_to_matrix(sleuth_object,
                             "obs_norm",
                             "est_counts") %>%
            as_tibble(rownames = "target_id") %>%
            mutate(across(-target_id, ~if_else(. == 0, . + pseudocount, .))) %>%
            pivot_longer(cols = -target_id,
                         names_to = "condition",
                         values_to = "value"
                        ) %>%
            left_join(
                metadata %>% 
                select(sample, !!sym(parameter)) %>%
                rename(condition = sample, group = !!sym(parameter)),
                by = "condition"
            ) %>%
            group_by(target_id, group) %>%
            summarize(mean = mean(value),
                      sd = sd(value),
                      .groups = 'drop'
                     ) %>%
            pivot_wider(names_from = group,
                        values_from = c(mean, sd),
                        names_sep = "_"
                       ) %>%
            mutate(
                mean_log2fc = log2(!!sym(paste0("mean_", groups[1])) / !!sym(paste0("mean_", groups[2]))),
                sd_log2fc = log2(!!sym(paste0("sd_", groups[1])) / !!sym(paste0("sd_", groups[2])))
            ) %>%
            left_join(wald_results %>%
                      rename_with(~paste0("wt_", .), -target_id),
                      by = "target_id") %>%
            left_join(lrt_results %>%
                      rename_with(~paste0("lrt_", .), -target_id),
                      by = "target_id") %>%
            arrange(str_extract(target_id, "^[^0-9]+"),
                    as.numeric(str_extract(target_id, "\\d+"))
            )

## save plotting table

plotting %>% write_tsv(plotting_filename)

In [37]:
## now plot the data with colouring for significance and fold-change

plotting <- plotting %>% mutate(wt_sig = ifelse(wt_qval <= alpha &
                                                abs(mean_log2fc) >= log2(FC_threshold),
                                                TRUE, FALSE)) %>%
                         mutate(lrt_sig = ifelse(lrt_qval <= alpha &
                                                 abs(mean_log2fc) >= log2(FC_threshold),
                                                 TRUE, FALSE)) %>%
                         select(target_id, mean_log2fc, wt_qval, lrt_qval, wt_sig, lrt_sig)

In [20]:
## set up aesthetics

text_theme <- theme(
              #legend.position="none",
              panel.grid.major = element_blank(),
              panel.grid.minor = element_blank(),
              legend.text=element_text(family = "Helvetica", size = 15, colour = "black", face = "bold"),
              legend.title=element_text(family = "Helvetica", size = 15, colour = "black", face = "bold"),
              #legend.title=element_blank(),
              legend.key=element_blank(),
              strip.text.x = element_text(size = 15, face = "bold"),
              plot.title = element_text(family = "Helvetica", size = 15, colour = "black", face = "bold", hjust = 0.5),
              axis.title = element_text(family = "Helvetica", size = 20, colour = "black", face = "bold"),
              axis.title.y = element_text(margin = margin(t = 0, r = 5, b = 0, l = 0), size = 18, face = "bold"),
              axis.title.x = element_text(margin = margin(t = 10, r = 0, b = 0, l = 0), size = 18,face = "bold"
                                         ),
              axis.text.y = element_text(family = "Helvetica",
                                         size=15, 
                                         face="bold", 
                                         color = "black"),
              axis.text.x = element_text(family = "Helvetica", 
                                         size = 15, 
                                         face = "bold",
                                         colour = "black",
                                         angle = 0,
                                         #vjust = 0.5,
                                         #hjust=1
                                        ),
              panel.background=element_rect(colour="black"),
              panel.border = element_rect(colour = "black", fill=NA, linewidth=2),
              axis.text = element_text(face="bold")
              )

In [21]:
## set canvas size

options(repr.plot.width = 6, repr.plot.height = 6)

In [22]:
## dynamic plotting function (derived from my manual code by Claude)

create_volcano_plot <- function(
    plotting,
    alpha,
    FC_threshold,
    groups,
    fc_column = "mean_log2fc",
    pval_column = "wt_qval",
    sig_column = "wt_sig",
    plot_title = "Denim - Wald Test",
    text_size = 4
  ) {
  
  # Ensure columns exist
  required_cols <- c(fc_column, pval_column, sig_column)
  missing_cols <- !required_cols %in% colnames(plotting)
  if (any(missing_cols)) {
    stop(sprintf("Missing required columns: %s", 
                paste(required_cols[missing_cols], collapse = ", ")))
  }
  
  # Calculate plot boundaries
  max_abs_x <- max(abs(plotting[[fc_column]]))
  min_abs_y <- min(abs(plotting[[pval_column]]))
  max_y <- max(-log10(plotting[[pval_column]]))
  
  # Calculate relative positions for annotations
  x_margin <- max_abs_x * 0.1  # 10% of x-axis range
  y_margin <- max_y * 0.1      # 10% of y-axis range
  
  # Create base plot with dynamic column references
  p <- plotting %>% 
    ggplot(aes(
      x = .data[[fc_column]],
      y = -log10(.data[[pval_column]]),
      col = .data[[sig_column]]
    )) +
    geom_point(size = 3.5) +
    xlim(-max_abs_x, max_abs_x)
  
  # Add dynamic annotations
  p <- p +
    # Zero line
    geom_vline(xintercept = 0,
               linetype = "dotted", 
               color = "black",
               linewidth = 1) +
    
    # Q-value threshold line and label
    geom_hline(yintercept = -log10(alpha),
               linetype = "dashed", 
               color = "dark grey",
               linewidth = 1.0) +
    annotate("text",
             x = -max_abs_x,
             y = -log10(alpha) + y_margin / 1.5,
             label = bquote(atop("q-value >", -log[10]*"("*.(alpha)*")")),
             hjust = 0,
             fontface = "bold",
             size = text_size,
             color = "dark grey") +
    
    # FC threshold lines and labels
    geom_vline(xintercept = log2(FC_threshold),
               linetype = "dashed", 
               color = "dark grey",
               linewidth = 1) +
    geom_vline(xintercept = -log2(FC_threshold),
               linetype = "dashed", 
               color = "dark grey",
               linewidth = 1) +
    
    # Dynamic FC threshold labels
    annotate("text",
             x = log2(FC_threshold) / 1.3,
             y = y_margin * 1.5,
             label = bquote("FC" > .(sprintf("%.0f", FC_threshold))),
             hjust = -0.0,
             fontface = "bold",
             size = text_size,
             angle = 90,
             color = "dark grey") +
    annotate("text",
             x = -log2(FC_threshold) / 1.3,
             y = y_margin * 1.5,
             label = bquote("FC" < .(sprintf("%.0f", -FC_threshold))),
             hjust = -0.0,
             fontface = "bold",
             size = text_size,
             angle = 90,
             color = "dark grey") +
    
    # Dynamic arrows and enrichment labels
    geom_segment(aes(x = log2(FC_threshold) + x_margin,
                    xend = log2(FC_threshold) + (3 * x_margin),
                    y = max_y, 
                    yend = max_y), 
                arrow = arrow(length = unit(0.1, "inches")), 
                color = "dark grey") + 
    annotate("text",
             x = log2(FC_threshold) + x_margin,
             y = max_y - y_margin * 0.75,
             label = paste0("enriched in\n", groups[1]),
             hjust = 0.0,
             fontface = "bold",
             size = text_size,
             color = "dark grey") +
    
    geom_segment(aes(x = -log2(FC_threshold) - x_margin,
                    xend = -log2(FC_threshold) - (3 * x_margin),
                    y = max_y, 
                    yend = max_y), 
                arrow = arrow(length = unit(0.1, "inches")), 
                color = "dark grey") + 
    annotate("text",
             x = -log2(FC_threshold) - x_margin,
             y = max_y - y_margin * 0.75,
             label = paste("enriched in\n", groups[2]),
             hjust = 1.0,
             fontface = "bold",
             size = text_size,
             color = "dark grey") +
    
    # Theme and aesthetics
    scale_color_manual(values = c("FALSE" = "dark grey",
                                "TRUE" = "blue")) +
    labs(title = plot_title,
         x = expression("log"[2]*"(FC)"),
         y = expression("-log"[10]*"(q-value)"),
         color = "Significant") +
    theme_bw() +
    theme(aspect.ratio = 1,
          legend.position = "none") +
    text_theme
  
  return(p)
}

In [39]:
## plotting Wald results

p <- create_volcano_plot(plotting = plotting,
                         alpha = alpha,
                         FC_threshold = FC_threshold,
                         groups = groups,
                         fc_column = "mean_log2fc",
                         pval_column = "wt_qval",
                         sig_column = "wt_sig",
                         plot_title = "Denim - Wald Test",
                        )

## saving

ggsave(filename = wald_plot_filename,
       plot = p,
       dpi = 1200,
       width = 6,
       height = 6
      )

Warning message in geom_segment(aes(x = log2(FC_threshold) + x_margin, xend = log2(FC_threshold) + :
“All aesthetics have length 1, but the data has 656 rows.
ℹ Please consider using `annotate()` or provide this layer with
  data containing a single row.”
Warning message in geom_segment(aes(x = -log2(FC_threshold) - x_margin, xend = -log2(FC_threshold) - :
“All aesthetics have length 1, but the data has 656 rows.
ℹ Please consider using `annotate()` or provide this layer with
  data containing a single row.”
Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'language'”
Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'language'”
Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'language'”


In [40]:
## plotting LRT results

p <- create_volcano_plot(plotting = plotting,
                         alpha = alpha,
                         FC_threshold = FC_threshold,
                         groups = groups,
                         fc_column = "mean_log2fc",
                         pval_column = "lrt_qval",
                         sig_column = "lrt_sig",
                         plot_title = "Denim - Likelihood Ratio Test",
                        )

## saving

ggsave(filename = lrt_plot_filename,
       plot = p,
       dpi = 1200,
       width = 6,
       height = 6
      )

Warning message in geom_segment(aes(x = log2(FC_threshold) + x_margin, xend = log2(FC_threshold) + :
“All aesthetics have length 1, but the data has 656 rows.
ℹ Please consider using `annotate()` or provide this layer with
  data containing a single row.”
Warning message in geom_segment(aes(x = -log2(FC_threshold) - x_margin, xend = -log2(FC_threshold) - :
“All aesthetics have length 1, but the data has 656 rows.
ℹ Please consider using `annotate()` or provide this layer with
  data containing a single row.”
Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'language'”
Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'language'”
Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'language'”
